In [1]:
import os
import csv
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
import pickle as pkl
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Dropout
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [2]:
if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


In [3]:
csv_path = r"C:\Users\Agustin\Documents\RepositorioCEIA_Git\VpC2_Datasets\severstal-steel-defect-detection/train.csv"
train_path = r"C:\Users\Agustin\Documents\RepositorioCEIA_Git\VpC2_Datasets\severstal-steel-defect-detection/train_images/"
train_masks_path = r"C:\Users\Agustin\Documents\RepositorioCEIA_Git\VpC2_Datasets\severstal-steel-defect-detection/train_masks/"

In [4]:
def decode_pixels(pixels_str):
    """
    This function receives a list which contains the pixels as a space-separated code.
    For example, '1 3 10 5' implies pixels 1,2,3,10,11,12,13,14 are to be included in the mask
    """
    
    pixels_encoded_list = pixels_str.split(' ')
    pixels_list = []
    for i in range(0, len(pixels_encoded_list), 2):
        # Add to the pixels list all the pixels encoded by the 2 digits
        pixels_list += range(
            int(pixels_encoded_list[i]) - 1,
            int(pixels_encoded_list[i]) + int(pixels_encoded_list[i + 1]) - 1,
        )
        
    # Return the decoded list    
    return pixels_list

In [5]:
train_csv = pd.read_csv(csv_path)

In [6]:
train_files = listdir(train_path)

In [7]:
n_h = 256
n_w = 1600
n_class = 5

In [8]:
for f in train_files:
    
    y_out=train_csv[train_csv['ImageId'].eq(f)]
    
    image_pixels = np.zeros((n_h, n_w), dtype=np.uint8)

    for index, row in y_out.iterrows():

        # Generate the mask of the error type
        mask = np.zeros((n_h * n_w, 1), dtype=np.uint8)
        mask[decode_pixels(row['EncodedPixels'])] = int(row['ClassId'])
        mask = mask.reshape(n_h, n_w)
    
        # Update channel of the given error
        image_pixels = image_pixels + mask
    
    cv2.imwrite(train_masks_path+ f + ".bmp", image_pixels)

In [9]:
def build_model(input_layer, start_neurons):
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(input_layer)
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)
    pool1 = Dropout(0.25)(pool1)

    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(pool1)
    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)
    pool2 = Dropout(0.5)(pool2)

    conv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(pool2)
    conv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)
    pool3 = Dropout(0.5)(pool3)

    conv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(pool3)
    conv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(conv4)
    pool4 = MaxPooling2D((2, 2))(conv4)
    pool4 = Dropout(0.5)(pool4)

    # Mitad
    convm = Conv2D(start_neurons * 16, (3, 3), activation="relu", padding="same")(pool4)
    convm = Conv2D(start_neurons * 16, (3, 3), activation="relu", padding="same")(convm)
    
    deconv4 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv4 = concatenate([deconv4, conv4])
    uconv4 = Dropout(0.5)(uconv4)
    uconv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(uconv4)
    uconv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(uconv4)

    deconv3 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(uconv4)
    uconv3 = concatenate([deconv3, conv3])
    uconv3 = Dropout(0.5)(uconv3)
    uconv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(uconv3)
    uconv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(uconv3)

    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv3)
    uconv2 = concatenate([deconv2, conv2])
    uconv2 = Dropout(0.5)(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)

    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = Dropout(0.5)(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)
    
    output_layer = Conv2D(5, (1,1), padding="same", activation="softmax")(uconv1)
    
    return output_layer

input_layer = Input((256, 1600, 3))
output_layer = build_model(input_layer, 16)

model = Model(input_layer, output_layer)

In [10]:
model.compile(loss=sparse_categorical_crossentropy, optimizer="adam", metrics=["accuracy"])

In [11]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 1600, 3 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 1600, 16 448         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 1600, 16 2320        conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 128, 800, 16) 0           conv2d_1[0][0]                   
_______________________________________________________________________________________

In [12]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    )

mask_datagen = ImageDataGenerator(
    )


image_generator = train_datagen.flow_from_directory(
        r'C:\Users\Agustin\Documents\RepositorioCEIA_Git\VpC2_Datasets\severstal-steel-defect-detection/',
        target_size=(n_h, n_w),
        batch_size=8,
        shuffle=False,
        color_mode="rgb",
        classes=['train_images'],
        class_mode=None)

mask_generator = mask_datagen.flow_from_directory(
        r'C:\Users\Agustin\Documents\RepositorioCEIA_Git\VpC2_Datasets\severstal-steel-defect-detection/',
        target_size=(n_h, n_w),
        batch_size=8,
        shuffle=False,
        color_mode="grayscale",
        classes=['train_masks'], 
        class_mode=None)

train_generator = zip(image_generator, mask_generator)

Found 12568 images belonging to 1 classes.
Found 12568 images belonging to 1 classes.


In [16]:
early_stopping = EarlyStopping(patience=10, verbose=1)
model_checkpoint = ModelCheckpoint("./keras.model", save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.00001, verbose=1)

epochs = 10
steps_per_epoch=int(len(train_files)/8)

history = model.fit(train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=epochs,
                    #callbacks=[early_stopping, model_checkpoint, reduce_lr])
                    )

Epoch 1/10
1571/1571 [==============================] - 981s 625ms/step - loss: 0.1622 - accuracy: 1.0000
Epoch 2/10
1571/1571 [==============================] - 970s 618ms/step - loss: 0.1599 - accuracy: 1.0000
Epoch 3/10
1571/1571 [==============================] - 958s 610ms/step - loss: 0.1595 - accuracy: 1.0000
Epoch 4/10
1571/1571 [==============================] - 955s 608ms/step - loss: 0.1590 - accuracy: 1.0000
Epoch 5/10
1571/1571 [==============================] - 967s 616ms/step - loss: 0.1583 - accuracy: 1.0000
Epoch 6/10
1571/1571 [==============================] - 947s 603ms/step - loss: 0.1578 - accuracy: 1.0000
Epoch 7/10
1571/1571 [==============================] - 942s 599ms/step - loss: 0.1573 - accuracy: 1.0000
Epoch 8/10
1571/1571 [==============================] - 946s 602ms/step - loss: 0.1566 - accuracy: 1.0000
Epoch 9/10
1571/1571 [==============================] - 946s 602ms/step - loss: 0.1561 - accuracy: 1.0000
Epoch 10/10
1084/1571 [===================>...

InternalError: GPU sync failed

In [17]:
model.save('./model')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


InternalError: GPU sync failed

In [ ]:
prediction=model.predict(image_generator[7])
np.max(np.argmax(prediction,axis=3))